# Capstone Project- The Battle of Neigbourhoods

# Table of contents

* Introduction
* Data
* Methodolgy
* Analysis
* Results and Discussion
* Conclusion


## Introduction

Being a Indian and Love for the Indian food allover the world is amazing but availability of Indian food and taste is not easy to find.

Switzerland is one of the famous tourist place and one of the favourite place for the Indians but the Availability of Indian food is not that much even if the location is in Downtown.

This Project will help us to find the location in switzerland (Zurich) and its nearby areas where Asian food can be available as Indian Food specifically will narrow our search.

Prior launching any restaurant, it’s important to know if the business as a good opportunity. In order to do so, this report will try to gather data about other restaurant localization, competitors and best localization.

These data could be use for a business plan afterward

## Problem

As the goal of this is to create a business plan in the end, we need to make sure data from api are correct. We also need to check that customer could be interested in this specific business.

In order to do so, a survey in Zurich will be done in addition to data gathering. I’ll go in the cities and check at different hours if restaurants are working, if streets are full and so on, and what kind of restaurant works well. This survey will allow to validate the data analysis done here.

## Interest 
This study can be used by anyone interested by opening a restaurant. Or any other business.

Maybe they will need to modify some data.

## Data

The idea of doing the project and data collection is taken from one of the tempalte shared in coursera project. I will keep the idea of clustering the city by area and then plot heatmap to find better area.

I will change some data:

Country/City: Switzerland
Goal: Open a restaurant/little shop for workers in weekday and maybe saturday
So, I will cross data from working days, and localisations.

I will use the following API:

Foursquare API: to find restaurant/venues
Google API: reverse geolocalisation

## Neighborhood Candidates

Firstly, we will  create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will be create a grid of cells covering our area of interest which is aprox. 1.5km killometers centered around Zurich city center.

Let's first find the latitude & longitude of Zurich city center, using specific, well known address and Google Maps geocoding API.

We'll consider the Prefecture to be the city center, as a lot of companies are around

### Importing all the librarires and paswrds

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math
import warnings

! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')


     |████████████████████████████████| 112kB 11.9MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 11.7MB/s eta 0:00:01
Folium installed
Libraries imported.


#### Let's assume that we are staying at the Zurich downtown (Central Plaza). So let's start by converting the Central Plaza address to its latitude and longitude coordinates.

In [2]:
address = 'Central 1, 8001 Zurich'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

47.3770836 8.543634


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~1.5km from Prefecture. Our neighborhoods will be defined as circular areas with a radius of 100 meters, so our neighborhood centers will be 200 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map

In [3]:
search_query = 'Asian food'
radius = 5000
print(search_query + ' .... OK!')

Asian food .... OK!


In [4]:
CLIENT_ID = 'M0XKWEK2RJZSE5XJIH0MPUBZPWZ2UGFUEGK5YCO1HGVGNGSU' # your Foursquare ID
CLIENT_SECRET = 'YMYYBKNJXFSPZ5XX1BMHJIZC0IDKVDYHFLODBWVHHCJALL5F' # your Foursquare Secret
ACCESS_TOKEN = 'DNGKZPAWP3VZZTOT0LXWDZGEEQXRQJZIJ0TU0NKOUBMOVLWD' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 40
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=M0XKWEK2RJZSE5XJIH0MPUBZPWZ2UGFUEGK5YCO1HGVGNGSU&client_secret=YMYYBKNJXFSPZ5XX1BMHJIZC0IDKVDYHFLODBWVHHCJALL5F&ll=47.3770836,8.543634&oauth_token=DNGKZPAWP3VZZTOT0LXWDZGEEQXRQJZIJ0TU0NKOUBMOVLWD&v=20180604&query=Asian food&radius=5000&limit=40'

In [5]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6074a08a97f3712659b4196e'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4e5407058877863330aa8874',
    'name': 'Asian Sport Center',
    'location': {'lat': 47.3826760023631,
     'lng': 8.547707985902278,
     'labeledLatLngs': [{'label': 'display',
       'lat': 47.3826760023631,
       'lng': 8.547707985902278}],
     'distance': 694,
     'cc': 'CH',
     'city': 'Zürich',
     'state': 'Zürich',
     'country': 'Schweiz',
     'formattedAddress': ['Zürich']},
    'categories': [{'id': '4bf58dd8d48988d1b2941735',
      'name': 'College Gym',
      'pluralName': 'College Gyms',
      'shortName': 'Gym',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1618256010',
    'hasPerk': False},
   {'id': '4ec4eefbf790c16e5225be61',
    'name': 'Fo Ke Kio Asian Take Away',
    'location'

## We will convert this to Pandas dataframe 

In [6]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head(20)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.crossStreet,location.postalCode,location.neighborhood,venuePage.id
0,4e5407058877863330aa8874,Asian Sport Center,"[{'id': '4bf58dd8d48988d1b2941735', 'name': 'C...",v-1618256010,False,47.382676,8.547708,"[{'label': 'display', 'lat': 47.3826760023631,...",694,CH,Zürich,Zürich,Schweiz,[Zürich],NaN,NaN,NaN,NaN,NaN
1,4ec4eefbf790c16e5225be61,Fo Ke Kio Asian Take Away,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1618256010,False,47.372142,8.534841,"[{'label': 'display', 'lat': 47.37214184335466...",861,CH,Zürich,Zürich,Schweiz,"[Talacker 50A, Zürich]",Talacker 50A,NaN,NaN,NaN,NaN
2,52013213498e9178ab9e16af,Little Nooch Asian Streetfood,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1618256010,False,47.411235,8.543783,"[{'label': 'display', 'lat': 47.41123460919642...",3801,CH,Zürich,Zürich,Schweiz,"[Hofwiesenstr. 369 (Bahnhof Oerlikon), 8050 Zü...",Hofwiesenstr. 369,Bahnhof Oerlikon,8050,NaN,NaN
3,4bf6cf4b4a67c9282a7322cf,Asian Moon,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1618256010,False,47.363043,8.549531,"[{'label': 'display', 'lat': 47.36304278640203...",1625,CH,Zürich,Zürich,Schweiz,"[Seefeldstrasse 40 (at Kreuzstrasse), 8008 Zür...",Seefeldstrasse 40,at Kreuzstrasse,8008,NaN,NaN
4,5193c15b498ec560cd705c47,Asian-Wellness,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",v-1618256010,False,47.391070,8.538933,"[{'label': 'display', 'lat': 47.39107015696727...",1596,CH,Zürich,Zürich,Schweiz,"[Schaffhauserstrasse 43, 8037 Zürich]",Schaffhauserstrasse 43,NaN,8037,NaN,NaN
5,5e0b4830456d3b000815955b,Viet Street Food,"[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",v-1618256010,False,47.376122,8.536371,"[{'label': 'display', 'lat': 47.376122, 'lng':...",557,CH,Zürich,Zürich,Schweiz,"[Usteristrasse 21, Zürich]",Usteristrasse 21,NaN,NaN,NaN,NaN
6,4f5bb376e4b0090c09cda75b,YEN'S Sushi & Asian Cuisine,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1618256010,False,47.371369,8.528704,"[{'label': 'display', 'lat': 47.37136901098179...",1292,CH,Zürich,Zürich,Schweiz,"[Hallwylerstr. 43, 8004 Zürich]",Hallwylerstr. 43,NaN,8004,NaN,NaN
7,4b59ca7af964a520369828e3,India Street-Food,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1618256010,False,47.383168,8.530197,"[{'label': 'display', 'lat': 47.383168, 'lng':...",1218,CH,Zürich,Zürich,Schweiz,"[Langstrasse 213, 8005 Zürich]",Langstrasse 213,NaN,8005,NaN,NaN
8,5357cc16498e0e40db63f54a,Jelmoli Food Market,"[{'id': '50be8ee891d4fa8dcc7199a7', 'name': 'M...",v-1618256010,False,47.374083,8.537229,"[{'label': 'display', 'lat': 47.3740827107709,...",587,CH,NaN,NaN,Schweiz,NaN,NaN,NaN,NaN,NaN,NaN
9,595a8086c365882ea49c863c,Asianart,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1618256010,False,47.377080,8.523304,"[{'label': 'display', 'lat': 47.37708, 'lng': ...",1532,CH,Zürich,Zürich,Schweiz,"[Schreinerstrasse 64, 8004 Zürich]",Schreinerstrasse 64,NaN,8004,NaN,NaN


#### Filtering the data frame

In [7]:
## keeping only the data frames that is related to venues and locations
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.reindex(columns = filtered_columns)

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else: 
        return(categories_list)[0]['name']
    
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,crossStreet,postalCode,neighborhood,id
0,Asian Sport Center,College Gym,47.382676,8.547708,"[{'label': 'display', 'lat': 47.3826760023631,...",694,CH,Zürich,Zürich,Schweiz,[Zürich],NaN,NaN,NaN,NaN,4e5407058877863330aa8874
1,Fo Ke Kio Asian Take Away,Asian Restaurant,47.372142,8.534841,"[{'label': 'display', 'lat': 47.37214184335466...",861,CH,Zürich,Zürich,Schweiz,"[Talacker 50A, Zürich]",Talacker 50A,NaN,NaN,NaN,4ec4eefbf790c16e5225be61
2,Little Nooch Asian Streetfood,Asian Restaurant,47.411235,8.543783,"[{'label': 'display', 'lat': 47.41123460919642...",3801,CH,Zürich,Zürich,Schweiz,"[Hofwiesenstr. 369 (Bahnhof Oerlikon), 8050 Zü...",Hofwiesenstr. 369,Bahnhof Oerlikon,8050,NaN,52013213498e9178ab9e16af
3,Asian Moon,Chinese Restaurant,47.363043,8.549531,"[{'label': 'display', 'lat': 47.36304278640203...",1625,CH,Zürich,Zürich,Schweiz,"[Seefeldstrasse 40 (at Kreuzstrasse), 8008 Zür...",Seefeldstrasse 40,at Kreuzstrasse,8008,NaN,4bf6cf4b4a67c9282a7322cf
4,Asian-Wellness,Spa,47.391070,8.538933,"[{'label': 'display', 'lat': 47.39107015696727...",1596,CH,Zürich,Zürich,Schweiz,"[Schaffhauserstrasse 43, 8037 Zürich]",Schaffhauserstrasse 43,NaN,8037,NaN,5193c15b498ec560cd705c47
5,Viet Street Food,Vietnamese Restaurant,47.376122,8.536371,"[{'label': 'display', 'lat': 47.376122, 'lng':...",557,CH,Zürich,Zürich,Schweiz,"[Usteristrasse 21, Zürich]",Usteristrasse 21,NaN,NaN,NaN,5e0b4830456d3b000815955b
6,YEN'S Sushi & Asian Cuisine,Sushi Restaurant,47.371369,8.528704,"[{'label': 'display', 'lat': 47.37136901098179...",1292,CH,Zürich,Zürich,Schweiz,"[Hallwylerstr. 43, 8004 Zürich]",Hallwylerstr. 43,NaN,8004,NaN,4f5bb376e4b0090c09cda75b
7,India Street-Food,Indian Restaurant,47.383168,8.530197,"[{'label': 'display', 'lat': 47.383168, 'lng':...",1218,CH,Zürich,Zürich,Schweiz,"[Langstrasse 213, 8005 Zürich]",Langstrasse 213,NaN,8005,NaN,4b59ca7af964a520369828e3
8,Jelmoli Food Market,Market,47.374083,8.537229,"[{'label': 'display', 'lat': 47.3740827107709,...",587,CH,NaN,NaN,Schweiz,NaN,NaN,NaN,NaN,NaN,5357cc16498e0e40db63f54a
9,Asianart,Chinese Restaurant,47.377080,8.523304,"[{'label': 'display', 'lat': 47.37708, 'lng': ...",1532,CH,Zürich,Zürich,Schweiz,"[Schreinerstrasse 64, 8004 Zürich]",Schreinerstrasse 64,NaN,8004,NaN,595a8086c365882ea49c863c


## Lets generate the Map with the help of folium

In [8]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start =15)

#To locate the central plaza, using red mark on the map
folium.CircleMarker(
    [latitude, longitude],
    radius=20,
    color = 'red',
    popup='Central Plaza',
    fill = 'True',
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# To locate the Indian resturant, using green mark on the map
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        color = 'green',
        popup = label,
        fill = 'True'
).add_to(venues_map)

venues_map

## Lets go to our near hotels of Zurich

In [9]:
#Now the above Map has all the resturants in Zurich, Now we will Locate the nearby hotels and will visulaize

search_query = 'Hotels'
radius = 5000
print(search_query + ' .... OK!')


Hotels .... OK!


In [10]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=M0XKWEK2RJZSE5XJIH0MPUBZPWZ2UGFUEGK5YCO1HGVGNGSU&client_secret=YMYYBKNJXFSPZ5XX1BMHJIZC0IDKVDYHFLODBWVHHCJALL5F&ll=47.3770836,8.543634&oauth_token=DNGKZPAWP3VZZTOT0LXWDZGEEQXRQJZIJ0TU0NKOUBMOVLWD&v=20180604&query=Hotels&radius=5000&limit=40'

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6074a0a05b8470476ef894b3'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4f192576e4b00583e7aecae5',
    'name': 'Chreis 4 Hotels / Gastenhaus',
    'location': {'lat': 47.377868,
     'lng': 8.528536,
     'labeledLatLngs': [{'label': 'display',
       'lat': 47.377868,
       'lng': 8.528536}],
     'distance': 1141,
     'cc': 'CH',
     'country': 'Schweiz'},
    'categories': [],
    'referralId': 'v-1618256032',
    'hasPerk': False},
   {'id': '4f9b25dce4b020e60ecae7e7',
    'name': 'Chreis Hotels 4, Lodges',
    'location': {'address': 'Anwandstrasse 10',
     'lat': 47.37620385434355,
     'lng': 8.524694040843906,
     'labeledLatLngs': [{'label': 'display',
       'lat': 47.37620385434355,
       'lng': 8.524694040843906}],
     'distance': 1431,
     'cc': 'CH',
     'city': 'Zürich',
     'state': 'Zürich',
     'country': 'Schweiz',
     'formattedAddress': ['Anwandstrass

In [12]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.address,location.city,location.state,location.formattedAddress,location.postalCode,venuePage.id,location.crossStreet
0,4f192576e4b00583e7aecae5,Chreis 4 Hotels / Gastenhaus,[],v-1618256032,False,47.377868,8.528536,"[{'label': 'display', 'lat': 47.377868, 'lng':...",1141,CH,Schweiz,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4f9b25dce4b020e60ecae7e7,"Chreis Hotels 4, Lodges","[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1618256032,False,47.376204,8.524694,"[{'label': 'display', 'lat': 47.37620385434355...",1431,CH,Schweiz,Anwandstrasse 10,Zürich,Zürich,"[Anwandstrasse 10, Zürich]",NaN,NaN,NaN
2,4b058887f964a520b8cb22e3,Hotel Schweizerhof Zürich,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1618256032,False,47.377046,8.539562,"[{'label': 'display', 'lat': 47.37704590573849...",306,CH,Schweiz,Bahnhofplatz 7,Zürich,Zürich,"[Bahnhofplatz 7, 8001 Zürich]",8001,NaN,NaN
3,4e58f05db0fbdca309f5e43f,Hotel Scheuble,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1618256032,False,47.374447,8.544851,"[{'label': 'display', 'lat': 47.37444716313871...",307,CH,Schweiz,NaN,Zürich,Zürich,[Zürich],NaN,NaN,NaN
4,5d88688c229a2400089f4135,Hotel Schweizerhof Fitness Center,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1618256032,False,47.376946,8.539385,"[{'label': 'display', 'lat': 47.376946, 'lng':...",320,CH,Schweiz,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
## keeping only the data frames that is related to venues and locations
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
df_fil_hotel = dataframe.reindex(columns = filtered_columns)

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else: 
        return(categories_list)[0]['name']
    
# filter the category for each row
df_fil_hotel['categories'] = df_fil_hotel.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_fil_hotel.columns = [column.split('.')[-1] for column in df_fil_hotel.columns]

df_fil_hotel.head()

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,address,city,state,formattedAddress,postalCode,crossStreet,id
0,Chreis 4 Hotels / Gastenhaus,None,47.377868,8.528536,"[{'label': 'display', 'lat': 47.377868, 'lng':...",1141,CH,Schweiz,NaN,NaN,NaN,NaN,NaN,NaN,4f192576e4b00583e7aecae5
1,"Chreis Hotels 4, Lodges",Hotel,47.376204,8.524694,"[{'label': 'display', 'lat': 47.37620385434355...",1431,CH,Schweiz,Anwandstrasse 10,Zürich,Zürich,"[Anwandstrasse 10, Zürich]",NaN,NaN,4f9b25dce4b020e60ecae7e7
2,Hotel Schweizerhof Zürich,Hotel,47.377046,8.539562,"[{'label': 'display', 'lat': 47.37704590573849...",306,CH,Schweiz,Bahnhofplatz 7,Zürich,Zürich,"[Bahnhofplatz 7, 8001 Zürich]",8001,NaN,4b058887f964a520b8cb22e3
3,Hotel Scheuble,Hotel,47.374447,8.544851,"[{'label': 'display', 'lat': 47.37444716313871...",307,CH,Schweiz,NaN,Zürich,Zürich,[Zürich],NaN,NaN,4e58f05db0fbdca309f5e43f
4,Hotel Schweizerhof Fitness Center,Gym,47.376946,8.539385,"[{'label': 'display', 'lat': 47.376946, 'lng':...",320,CH,Schweiz,NaN,NaN,NaN,NaN,NaN,NaN,5d88688c229a2400089f4135


In [14]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start =15)

#To locate the central plaza, using red mark on the map
folium.CircleMarker(
    [latitude, longitude],
    radius=20,
    color = 'red',
    popup='Central Plaza',
    fill = 'True',
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# To locate the Hotels, using green mark on the map
for lat, lng, label in zip(df_fil_hotel.lat, df_fil_hotel.lng, df_fil_hotel.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        color = 'green',
        popup = label,
        fill = 'True'
).add_to(venues_map)

venues_map

#### Now we will create global map hotels vs Asian Resturants

In [15]:
Global_Map = folium.Map(location=[latitude, longitude], zoom_start =15)

#To locate the central plaza, using red mark on the map
folium.CircleMarker(
    [latitude, longitude],
    radius=20,
    color = 'red',
    popup='Central Plaza',
    fill = 'True',
    fill_color = 'Yellow',
    fill_opacity = 0.6
).add_to(Global_Map)

# To locate the Hotels, using green mark on the map
for lat, lng, label in zip(df_fil_hotel.lat, df_fil_hotel.lng, df_fil_hotel.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        color = 'green',
        popup = label,
        fill = 'True'
).add_to(Global_Map)
# To locate the Asian Resturants, using Red mark on the map
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        color = 'red',
        popup = label,
        fill = 'True'
).add_to(Global_Map)


Global_Map

###### We will now focus on spotting worker area more than competitors, we will try do determine where are clusters more of the worker.
In order to do that, we will use the k mean clustering method.

In [16]:
# Importing librariers for Clustering
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser

%matplotlib inline 
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')

Libraries imported.


In [17]:
### Creating a dataframe to perform clustering

In [18]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6074a0ab2437f20228332b55'},
 'response': {'venues': []}}

In [19]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [20]:
trending_venues_df

'No trending venues are available at the moment!'

In [21]:
import numpy as np
import pandas as pd
import json

import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
!pip install html5lib
!pip install lxml
!pip install BeautifulSoup4

print('librariers are imported')

     |████████████████████████████████| 122kB 10.5MB/s eta 0:00:01
librariers are imported


In [22]:
from pandas.io.html import read_html
url ='https://www.geonames.org/postalcode-search.html?q=&country=CH&adminCode1=ZH'
Zurich_canton = read_html(url)
print('Data downloaded!')

Data downloaded!


In [23]:
Zurich_canton[2]

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,ZÃ¼rich,8001,Switzerland,Kanton ZÃ¼rich,Bezirk ZÃ¼rich,ZÃ¼rich
1,NaN,47.367/8.55,47.367/8.55,47.367/8.55,47.367/8.55,47.367/8.55,47.367/8.55
2,2.0,ZÃ¼rich,8002,Switzerland,Kanton ZÃ¼rich,Bezirk ZÃ¼rich,ZÃ¼rich
3,NaN,47.367/8.55,47.367/8.55,47.367/8.55,47.367/8.55,47.367/8.55,47.367/8.55
4,3.0,ZÃ¼rich,8003,Switzerland,Kanton ZÃ¼rich,Bezirk ZÃ¼rich,ZÃ¼rich
...,...,...,...,...,...,...,...
396,199.0,Ebertswil,8925,Switzerland,Kanton ZÃ¼rich,Bezirk Affoltern,Hausen am Albis
397,NaN,47.226/8.55,47.226/8.55,47.226/8.55,47.226/8.55,47.226/8.55,47.226/8.55
398,200.0,Kappel am Albis,8926,Switzerland,Kanton ZÃ¼rich,Bezirk Affoltern,Kappel am Albis
399,NaN,47.228/8.527,47.228/8.527,47.228/8.527,47.228/8.527,47.228/8.527,47.228/8.527


In [24]:
df_zurich_Canton = Zurich_canton[2]

In [25]:
df_zurich_Canton.iloc[::2]

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,ZÃ¼rich,8001,Switzerland,Kanton ZÃ¼rich,Bezirk ZÃ¼rich,ZÃ¼rich
2,2.0,ZÃ¼rich,8002,Switzerland,Kanton ZÃ¼rich,Bezirk ZÃ¼rich,ZÃ¼rich
4,3.0,ZÃ¼rich,8003,Switzerland,Kanton ZÃ¼rich,Bezirk ZÃ¼rich,ZÃ¼rich
6,4.0,ZÃ¼rich,8004,Switzerland,Kanton ZÃ¼rich,Bezirk ZÃ¼rich,ZÃ¼rich
8,5.0,ZÃ¼rich,8005,Switzerland,Kanton ZÃ¼rich,Bezirk ZÃ¼rich,ZÃ¼rich
...,...,...,...,...,...,...,...
392,197.0,Aeugst am Albis,8914,Switzerland,Kanton ZÃ¼rich,Bezirk Affoltern,Aeugst am Albis
394,198.0,Hausen am Albis,8915,Switzerland,Kanton ZÃ¼rich,Bezirk Affoltern,Hausen am Albis
396,199.0,Ebertswil,8925,Switzerland,Kanton ZÃ¼rich,Bezirk Affoltern,Hausen am Albis
398,200.0,Kappel am Albis,8926,Switzerland,Kanton ZÃ¼rich,Bezirk Affoltern,Kappel am Albis


In [26]:
df_ZH_canton = df_zurich_Canton.iloc[::2]
df_ZH_canton

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,ZÃ¼rich,8001,Switzerland,Kanton ZÃ¼rich,Bezirk ZÃ¼rich,ZÃ¼rich
2,2.0,ZÃ¼rich,8002,Switzerland,Kanton ZÃ¼rich,Bezirk ZÃ¼rich,ZÃ¼rich
4,3.0,ZÃ¼rich,8003,Switzerland,Kanton ZÃ¼rich,Bezirk ZÃ¼rich,ZÃ¼rich
6,4.0,ZÃ¼rich,8004,Switzerland,Kanton ZÃ¼rich,Bezirk ZÃ¼rich,ZÃ¼rich
8,5.0,ZÃ¼rich,8005,Switzerland,Kanton ZÃ¼rich,Bezirk ZÃ¼rich,ZÃ¼rich
...,...,...,...,...,...,...,...
392,197.0,Aeugst am Albis,8914,Switzerland,Kanton ZÃ¼rich,Bezirk Affoltern,Aeugst am Albis
394,198.0,Hausen am Albis,8915,Switzerland,Kanton ZÃ¼rich,Bezirk Affoltern,Hausen am Albis
396,199.0,Ebertswil,8925,Switzerland,Kanton ZÃ¼rich,Bezirk Affoltern,Hausen am Albis
398,200.0,Kappel am Albis,8926,Switzerland,Kanton ZÃ¼rich,Bezirk Affoltern,Kappel am Albis


In [27]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 12.0MB/s ta 0:00:01


In [47]:
# Define latitude and longitude as lists.
latitude = []
longitude = []

# Loop through the City_State_Postal_Code data in the dataframe and for each postal code loop until coordicates are obtained.
for Code in df_ZH_canton['Code']:
    g = geocoder.arcgis(Code)
    print(Code, g.latlng)
    while(g.latlng is None):
        g = geocoder.arcgis(Code)
        print(Code, g.latlng)
    lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])

df_ZH_canton['Latitude'] = latitude
df_ZH_canton['Longitude'] = longitude

df_ZH_canton

8001 [39.55849500000005, -75.35896999999994]
8002 [39.942390000000046, -75.03218999999996]
8003 [39.90093150000007, -74.96151439999994]
8004 [39.76769000000007, -74.89097999999996]
8005 [39.77037270000005, -74.25673529999995]
8006 [39.75571000000008, -74.10758499999997]
8008 [39.56356000000005, -74.23974499999997]
8032 [39.782065000000046, -75.06427739699996]
8037 [39.63447000000008, -74.80602499999998]
8038 [39.50761000000006, -75.46219499999995]
8044 [39.76337500000005, 9.543418770000073]
8045 [39.868880700000034, -75.03791579999995]
8046 [40.01792500000005, -74.87969499999997]
8047 [39.683895000000064, 9.60137861100003]
8048 [39.97951500000005, -74.79789499999998]
8049 [39.85591500000004, -75.03513999999996]
8050 [39.72646000000003, -74.27838499999996]
8051 [39.79034500000006, -75.17560499999996]
8052 [39.95789000000008, -74.99921499999994]
8055 [39.86004500000007, -74.80268499999994]
8057 [39.96293000000003, -74.94886999999994]
8105 [39.944629500000076, -75.08741519999995]
8134 [50

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,Place,Code,Country,Latitude,Longitude
0,1.0,ZÃ¼rich,8001,Switzerland,39.558495,-75.358970
2,2.0,ZÃ¼rich,8002,Switzerland,39.942390,-75.032190
4,3.0,ZÃ¼rich,8003,Switzerland,39.900932,-74.961514
6,4.0,ZÃ¼rich,8004,Switzerland,39.767690,-74.890980
8,5.0,ZÃ¼rich,8005,Switzerland,39.770373,-74.256735
...,...,...,...,...,...,...
392,197.0,Aeugst am Albis,8914,Switzerland,42.609850,25.948016
394,198.0,Hausen am Albis,8915,Switzerland,42.578445,26.024073
396,199.0,Ebertswil,8925,Switzerland,47.228461,8.553584
398,200.0,Kappel am Albis,8926,Switzerland,47.225150,8.516513


In [48]:
def get_category_type(row):
    categories_list = row['Category']  
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [53]:
explore_df = pd.DataFrame([item for item in explore_df_list])
explore_df.columns = ['Place',
                      'Latitude', 
                      'Longitude',
                      'Name', 
                      'Category', 
                      'Lat', 
                      'Long']
explore_df.head()

ValueError: Length mismatch: Expected axis has 0 elements, new values have 7 elements

In [ ]:
dataframe_filtered

In [ ]:
name	categories	lat	lng	